In [1]:
import torch

In [2]:
def get_random_problems(batch_size, problem_size): # size를 입력으로 받아 무작위로 생성된 경로 문제를 반환
    #batch_size = 동시에 생성할 문제의 개수
    #problem_size = 문제 내의 고객 노드의 수(방문해야 할 위치의 수)

    depot_xy = torch.full((batch_size, 1, 2), 0.5) # 나중에 depot 고정해야하니까 잊지 말기!!!
    # shape: (batch, 1, 2) # (문제의 개수, depot의 개수, 2차원)

    node_xy = torch.rand(size=(batch_size, problem_size, 2))
    # shape: (batch, problem, 2)  # (문제의 개수, node의 개수, 2차원)

    if problem_size == 10:
        demand_scaler = 15
    elif problem_size == 20:
        demand_scaler = 30
    elif problem_size == 50:
        demand_scaler = 40
    elif problem_size == 100:
        demand_scaler = 50 ## 현재 problem size가 20 50 100으로 고정되어 있는데 이건 나중에 바꿔도 됨
    else:
        raise NotImplementedError

    node_demand = torch.randint(1, 10, size=(batch_size, problem_size)) / float(demand_scaler)
    # shape: (batch, problem) #demand scaler를 통해 수요의 크기가 문제의 크기에 맞게 조정

    return depot_xy, node_xy, node_demand

In [3]:
get_random_problems(1,10)

(tensor([[[0.5000, 0.5000]]]),
 tensor([[[0.1300, 0.8277],
          [0.4604, 0.1560],
          [0.8385, 0.0326],
          [0.7908, 0.5786],
          [0.7621, 0.3990],
          [0.3824, 0.2639],
          [0.3439, 0.5362],
          [0.5805, 0.9147],
          [0.0329, 0.5789],
          [0.7444, 0.0801]]]),
 tensor([[0.4667, 0.4000, 0.4000, 0.5333, 0.6000, 0.2667, 0.0667, 0.4667, 0.2667,
          0.2000]]))

In [4]:
def augment_xy_data_by_8_fold(xy_data):
    # xy_data.shape: (batch, N, 2) # 입력된 좌표 데이터를 8배로 증강하여 반환
    # N은 각 문제에 포함된 노드의 수를 의미

    x = xy_data[:, :, [0]] #모든 (x,y) 좌표중 x값만 뽑아냄
    y = xy_data[:, :, [1]] #모든 (x,y) 좌표중 y값만 뽑아냄
    # x,y shape: (batch, N, 1)

    dat1 = torch.cat((x, y), dim=2) #원본 데이터
    dat2 = torch.cat((1 - x, y), dim=2) # x축 기준 좌표 반전
    dat3 = torch.cat((x, 1 - y), dim=2) # y축 기준 좌표 반전
    dat4 = torch.cat((1 - x, 1 - y), dim=2) # 원점 기준 좌표 반전
    dat5 = torch.cat((y, x), dim=2) # y=x 직선에 대한 반전
    dat6 = torch.cat((1 - y, x), dim=2) # y=-x 직선에 대한 반전
    dat7 = torch.cat((y, 1 - x), dim=2) # x 축 반전 후 y= x에 대해 반전
    dat8 = torch.cat((1 - y, 1 - x), dim=2) # 원점 기준 좌표 반전 후 y = x 직선에 대해 반전

    aug_xy_data = torch.cat((dat1, dat2, dat3, dat4, dat5, dat6, dat7, dat8), dim=0)
    # shape: (8*batch, N, 2)
    # 한 데이터로 8개의 데이터를 만듦으로 총 데이터의 shape 은 (8*B, N, 2)
 
    return aug_xy_data

In [6]:
augment_xy_data_by_8_fold(torch.tensor([# 예를 들어, batch=4, N=5일 경우 xy_data는 다음과 같은 데이터일 수 있음
    [[0.2, 0.4], [0.3, 0.5], [0.6, 0.7], [0.8, 0.9], [0.1, 0.3]],  # 첫 번째 배치의 노드 좌표
    [[0.1, 0.2], [0.3, 0.4], [0.5, 0.6], [0.7, 0.8], [0.9, 0.0]],  # 두 번째 배치
    [[0.4, 0.3], [0.6, 0.2], [0.8, 0.5], [0.9, 0.7], [0.1, 0.9]],  # 세 번째 배치
    [[0.2, 0.8], [0.4, 0.3], [0.5, 0.7], [0.9, 0.1], [0.8, 0.6]]   # 네 번째 배치
]))

tensor([[[0.2000, 0.4000],
         [0.3000, 0.5000],
         [0.6000, 0.7000],
         [0.8000, 0.9000],
         [0.1000, 0.3000]],

        [[0.1000, 0.2000],
         [0.3000, 0.4000],
         [0.5000, 0.6000],
         [0.7000, 0.8000],
         [0.9000, 0.0000]],

        [[0.4000, 0.3000],
         [0.6000, 0.2000],
         [0.8000, 0.5000],
         [0.9000, 0.7000],
         [0.1000, 0.9000]],

        [[0.2000, 0.8000],
         [0.4000, 0.3000],
         [0.5000, 0.7000],
         [0.9000, 0.1000],
         [0.8000, 0.6000]],

        [[0.8000, 0.4000],
         [0.7000, 0.5000],
         [0.4000, 0.7000],
         [0.2000, 0.9000],
         [0.9000, 0.3000]],

        [[0.9000, 0.2000],
         [0.7000, 0.4000],
         [0.5000, 0.6000],
         [0.3000, 0.8000],
         [0.1000, 0.0000]],

        [[0.6000, 0.3000],
         [0.4000, 0.2000],
         [0.2000, 0.5000],
         [0.1000, 0.7000],
         [0.9000, 0.9000]],

        [[0.8000, 0.8000],
         [0.60